In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import interact
import ipywidgets as widgets
import numpy as np
#frameNum 1부터 1씩 증가시키게 정렬
data = pd.read_csv("/content/scenario2_2.csv")
unique_frame_numbers = {frame: idx + 1 for idx, frame in enumerate(data['frameNum'].unique())}
data['frameNum'] = data['frameNum'].map(unique_frame_numbers)

#한 프레임 내에서 클러스터링
clustered_data = pd.DataFrame(columns = data.columns)
for frame, group in data.groupby("frameNum"):
  group_data = group[["Range", "Azimuth", "Elevation"]]
  scaler = StandardScaler()
  scaled_data = scaler.fit_transform(group_data)

  dbscan = DBSCAN(eps=1, min_samples=3)
  clusters = dbscan.fit_predict(scaled_data)
  group['cluster'] = clusters
  group = group.sort_values(by='cluster')
  clustered_data = pd.concat([clustered_data, group])

clustered_data = clustered_data[clustered_data['cluster'] != -1].reset_index().drop(columns='index')
clustered_data['x'] = clustered_data['Range'] * np.cos(clustered_data['Elevation']) * np.cos(clustered_data['Azimuth'])
clustered_data['y'] = clustered_data['Range'] * np.cos(clustered_data['Elevation']) * np.sin(clustered_data['Azimuth'])
clustered_data['z'] = clustered_data['Range'] * np.sin(clustered_data['Elevation'])

def plot_frame(frame):
  fig = plt.figure(figsize=(8, 8))
  ax = fig.add_subplot(111, projection='3d')
  clustered_data_frame = clustered_data[clustered_data['frameNum'] == frame]
  df_scale = clustered_data_frame[["x", "y", "z", "cluster"]]
  scatter = ax.scatter(
      df_scale['x'],
      df_scale['y'],
      df_scale['z'],
      c=df_scale['cluster'],
      cmap='viridis',
      marker='o',
      s=50)

  legend = ax.legend(*scatter.legend_elements(), title="Clusters")
  ax.add_artist(legend)
  ax.set_xlabel('x')
  ax.set_ylabel('y')
  ax.set_zlabel('z')
  ax.set_title(f'Frame {frame}')
  plt.show()

#슬라이더
frame_slider = widgets.IntSlider(
    value=1,
    min=clustered_data["frameNum"].min(),
    max=clustered_data["frameNum"].max(),

    step=1,
    description='Frame'
)
interact(plot_frame, frame=frame_slider)

<ipython-input-33-7c9e9a8562ae>:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  clustered_data = pd.concat([clustered_data, group])


interactive(children=(IntSlider(value=1, description='Frame', max=43, min=1), Output()), _dom_classes=('widget…

<function __main__.plot_frame(frame)>

In [ ]:
clustered_data.to_csv("data_with_xyz.xlsx")

In [ ]:
clustered_data

,frameNum,pointNum,Range,Azimuth,Elevation,Doppler,SNR,cluster,x,y,z
0,1,0,2.46575,-1.17,0.07,0.06972,9.240000,0.0,0.959661,-2.264781,0.172462
1,1,61,2.59225,-0.63,0.28,0.06972,7.120000,0.0,2.013036,-1.467734,0.716383
2,1,60,2.52900,-0.63,0.25,-0.20888,27.119999,0.0,1.979974,-1.443628,0.625685
3,1,59,2.46575,-0.63,0.25,-0.20888,52.639999,0.0,1.930455,-1.407523,0.610036
4,1,58,2.27625,-0.61,0.17,-0.20888,30.999999,0.0,1.838829,-1.285192,0.385101
...,...,...,...,...,...,...,...,...,...,...,...
3564,43,76,3.79350,0.60,0.12,0.55664,18.120000,1.0,3.108395,2.126568,0.454128
3565,43,77,3.85675,0.60,0.12,0.48720,6.360000,1.0,3.160222,2.162024,0.461700
3566,43,78,3.60400,0.61,0.15,0.55664,26.999999,1.0,2.920841,2.041431,0.538575
3567,43,70,3.66725,0.59,0.15,0.20888,34.639999,1.0,3.013050,2.017404,0.548027


In [ ]:
import numpy as np

def kth_smallest_value(matrix, k):
    flattened = np.ravel(matrix)
    sorted_values = np.sort(flattened)
    return sorted_values[k]

from scipy.spatial import distance
clustered_data_agg = clustered_data.groupby(['frameNum', 'cluster']).agg({'frameNum': 'first',
                                                                          'cluster': 'first',
                                                                          'x': 'mean',
                                                                          'y': 'mean',
                                                                          'z': 'mean',
                                                                          }).reset_index(drop=True)
global_clustered_data = clustered_data_agg[clustered_data_agg['frameNum'] == 1]
global_clustered_data['global_cluster'] = global_clustered_data['cluster']
frame_len = len(clustered_data_agg['frameNum'].unique())
generated_cluster_n = int(global_clustered_data['global_cluster'].max()) + 1

for i in range(1, frame_len):
  current_f = global_clustered_data[global_clustered_data['frameNum'] == i].reset_index().drop(columns='index')
  next_f = clustered_data_agg[clustered_data_agg['frameNum'] == i + 1].reset_index().drop(columns='index')
  used_current_f = [0 for i in range(len(current_f))]
  distance_matrix = distance.cdist(current_f[['x', 'y', 'z']], next_f[['x', 'y', 'z']], metric='euclidean')
  for j in range(distance_matrix.size):
    min_value = kth_smallest_value(distance_matrix, j)
    [row], [col] = np.where(distance_matrix == min_value)
    if (used_current_f[row] == 1):
      continue
    used_current_f[row] = 1
    if (min_value >= 3.5):
      next_f.loc[col, 'global_cluster'] = None
    else:
      next_f.loc[col, 'global_cluster'] = current_f.loc[row, 'global_cluster']
  g_arr = [i for i in range (generated_cluster_n, generated_cluster_n + next_f['global_cluster'].isna().sum())]
  generated_cluster_n = generated_cluster_n + next_f['global_cluster'].isna().sum()
  next_f.loc[next_f['global_cluster'].isna(), 'global_cluster'] = g_arr[:]
  global_clustered_data = pd.concat([global_clustered_data, next_f]).reset_index().drop(columns='index')


<ipython-input-35-3fcaf0f50b78>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  global_clustered_data['global_cluster'] = global_clustered_data['cluster']


In [ ]:
global_clustered_data

,frameNum,cluster,x,y,z,global_cluster
0,1,0.0,1.705726,-1.601542,0.410367,0.0
1,1,1.0,4.184639,1.469327,0.343905,1.0
2,1,2.0,2.834504,1.657704,0.651839,2.0
3,2,0.0,1.887561,-1.477973,0.454277,0.0
4,2,1.0,4.148217,1.509764,0.703590,1.0
...,...,...,...,...,...,...
89,41,1.0,2.863386,1.968592,0.513841,19.0
90,42,0.0,2.069335,-1.096709,0.559938,17.0
91,42,1.0,3.051533,1.883823,0.552724,19.0
92,43,0.0,2.090437,-1.097339,0.675624,17.0


In [ ]:
global_clustered_data = pd.merge(clustered_data_agg, global_clustered_data[['cluster','frameNum','global_cluster']], on=['cluster', 'frameNum'], how='left')
global_clustered_data

,frameNum,cluster,x,y,z,global_cluster
0,1,0.0,1.705726,-1.601542,0.410367,0.0
1,1,1.0,4.184639,1.469327,0.343905,1.0
2,1,2.0,2.834504,1.657704,0.651839,2.0
3,2,0.0,1.887561,-1.477973,0.454277,0.0
4,2,1.0,4.148217,1.509764,0.703590,1.0
...,...,...,...,...,...,...
89,41,1.0,2.863386,1.968592,0.513841,19.0
90,42,0.0,2.069335,-1.096709,0.559938,17.0
91,42,1.0,3.051533,1.883823,0.552724,19.0
92,43,0.0,2.090437,-1.097339,0.675624,17.0


In [ ]:
clustered_data

,frameNum,pointNum,Range,Azimuth,Elevation,Doppler,SNR,cluster,x,y,z
0,1,0,2.46575,-1.17,0.07,0.06972,9.240000,0.0,0.959661,-2.264781,0.172462
1,1,61,2.59225,-0.63,0.28,0.06972,7.120000,0.0,2.013036,-1.467734,0.716383
2,1,60,2.52900,-0.63,0.25,-0.20888,27.119999,0.0,1.979974,-1.443628,0.625685
3,1,59,2.46575,-0.63,0.25,-0.20888,52.639999,0.0,1.930455,-1.407523,0.610036
4,1,58,2.27625,-0.61,0.17,-0.20888,30.999999,0.0,1.838829,-1.285192,0.385101
...,...,...,...,...,...,...,...,...,...,...,...
3564,43,76,3.79350,0.60,0.12,0.55664,18.120000,1.0,3.108395,2.126568,0.454128
3565,43,77,3.85675,0.60,0.12,0.48720,6.360000,1.0,3.160222,2.162024,0.461700
3566,43,78,3.60400,0.61,0.15,0.55664,26.999999,1.0,2.920841,2.041431,0.538575
3567,43,70,3.66725,0.59,0.15,0.20888,34.639999,1.0,3.013050,2.017404,0.548027


In [ ]:
clustered_data = pd.merge(clustered_data, global_clustered_data[['cluster','frameNum','global_cluster']], on=['cluster', 'frameNum'], how='left')

In [ ]:
clustered_data.drop(columns=['cluster'])

,frameNum,pointNum,Range,Azimuth,Elevation,Doppler,SNR,x,y,z,global_cluster
0,1,0,2.46575,-1.17,0.07,0.06972,9.240000,0.959661,-2.264781,0.172462,0.0
1,1,61,2.59225,-0.63,0.28,0.06972,7.120000,2.013036,-1.467734,0.716383,0.0
2,1,60,2.52900,-0.63,0.25,-0.20888,27.119999,1.979974,-1.443628,0.625685,0.0
3,1,59,2.46575,-0.63,0.25,-0.20888,52.639999,1.930455,-1.407523,0.610036,0.0
4,1,58,2.27625,-0.61,0.17,-0.20888,30.999999,1.838829,-1.285192,0.385101,0.0
...,...,...,...,...,...,...,...,...,...,...,...
3564,43,76,3.79350,0.60,0.12,0.55664,18.120000,3.108395,2.126568,0.454128,19.0
3565,43,77,3.85675,0.60,0.12,0.48720,6.360000,3.160222,2.162024,0.461700,19.0
3566,43,78,3.60400,0.61,0.15,0.55664,26.999999,2.920841,2.041431,0.538575,19.0
3567,43,70,3.66725,0.59,0.15,0.20888,34.639999,3.013050,2.017404,0.548027,19.0


In [ ]:
def plot_frame(frame):
  fig = plt.figure(figsize=(8, 8))
  ax = fig.add_subplot(111, projection='3d')
  clustered_data_frame = clustered_data[clustered_data['frameNum'] == frame]
  df_scale = clustered_data_frame[["x", "y", "z", "global_cluster"]]
  scatter = ax.scatter(
      df_scale['x'],
      df_scale['y'],
      df_scale['z'],
      c=df_scale['global_cluster'],
      cmap='viridis',
      marker='o',
      s=50)

  legend = ax.legend(*scatter.legend_elements(), title="Clusters")
  ax.add_artist(legend)
  ax.set_xlabel('x')
  ax.set_ylabel('y')
  ax.set_zlabel('z')
  ax.set_title(f'Frame {frame}')
  plt.show()
#슬라이더
frame_slider = widgets.IntSlider(
    value=1,
    min=clustered_data["frameNum"].min(),
    max=clustered_data["frameNum"].max(),
    step=1,
    description='Frame'
)
interact(plot_frame, frame=frame_slider)

interactive(children=(IntSlider(value=1, description='Frame', max=43, min=1), Output()), _dom_classes=('widget…

<function __main__.plot_frame(frame)>